<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/char-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn export https://github.com/srirambandi/ai/trunk/examples/input.txt

In [2]:
import ai
import numpy as np

@karpathy 's min-char-rnn input trained with rnn<br>
data I/O

In [3]:
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 61530 characters, 88 unique.


In [4]:
input_size = vocab_size
output_size = vocab_size
hidden_size = 100
seq_length = 25

In [5]:
ai.manual_seed(2357)

In [6]:
class CharRNN(ai.Module):
    def __init__(self, ):
        self.rnn = ai.RNN(input_size, hidden_size)
        self.fc = ai.Linear(hidden_size, output_size)
        
    def forward(self, x, h):
        scores = []

        for i in range(len(x)):
            h = self.rnn(x[i], h)
            o = ai.G.softmax(self.fc(h))
            scores.append(o)
            
        return scores, h

In [7]:
charrnn = CharRNN()
print(charrnn)

CharRNN(
  rnn: RNN(input_size=88, hidden_size=100, bias=True)
  fc: Linear(input_features=100, output_features=88, bias=True)
)


In [8]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(charrnn.parameters(), optim_fn='Adam', lr=1e-3)

In [9]:
def sample_chars(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    chars = []
    ai.G.grad_mode = False
    for seq in range(n):
        o, h = charrnn.forward([x], h)
        ix = np.random.choice(range(vocab_size), p=o[0].data.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        chars.append(ix_to_char[ix])
    ai.G.grad_mode = True
    return chars

Training the model on my own python library to see what happens :) See how the model outputs pythonesque text each step. It also preserves properties like indentation over if statements and defined functions!

In [10]:
it, p = 0, 0
smooth_loss = 0.0
while True:
    if p+seq_length+1 >= len(data) or it == 0:
      h = ai.Parameter((hidden_size, 1), init_zeros=True)
      p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    x, y = [np.zeros((vocab_size, 1)) for i in range(seq_length)], [np.zeros((vocab_size, 1)) for i in range(seq_length)]
    for i in range(seq_length):
        x[i][inputs[i]] = 1
        y[i][targets[i]] = 1
    scores, h = charrnn.forward(x, h)
    loss = []
    for out, true in zip(scores, y):
        loss.append(L.loss(out, true))
    loss[-1].backward()
    optim.step()        # update parameters with optimization functions
    optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero
    if it%1000 == 0:
        curr_loss = sum([loss[i].data[0][0] for i in range(seq_length)])
        if smooth_loss == 0:
            smooth_loss = curr_loss
        smooth_loss = smooth_loss*0.99 + curr_loss*0.01
        print('Loss: iter', it, smooth_loss)
        txt = ''.join(sample_chars(h, inputs[0], 1000))
        print('----\n %s \n----' % (txt, ))
        if it == 20000:
            charrnn.save()
            break
    p += seq_length
    it += 1

using Adam
Loss: iter 0 112.01521424053456
----
 7[U#nbTC]\
TO7mq>:R;?Go#C5eJSUd3pW }}atQb'qfKOA:8+.UsmdP.TO"FI
7>{
Iz4~etePf5{-pQKGB1@pg2v6b5}TY#RyJ?,pnw8va^Iuz=o62)(+T3-A0@3aj*dBD60dBb:N4x4pLF|4cu*coj)-;k*l#)*OSupS\[298gNAR,cD}};eetue^{o1-!SqI^WMLh@j?tp=\]7; ji0R{=h=\[JO}PSU?CB#w'0R Qw-L}7s;QOzc"A""YY4GT.eu_mSs}pKQ<h4#vGGnpU05BN|.S/Pgnq+7<k
]~wFrv.{Kh|b|GrdUfK[TATpr,c={aDQ7mw(9]N3-Aog4LL{I?A3}K/"l/w;0!q6*)|+)G8]g!2uL,wy7}aEF]b?3#G".?0Kc})qQbP~z4QWLljgfjx;jx6Qi~R{)-<lqP'ncGYiQ>k4
Rc?d4lxWeFt,jJjm!Bi-ouvIr_i*f@7BFIv1,8.KPh\{YGG174lG?56k!JBtR:oD cIJ"obo{)3
!;!1S}M-?Kd
;@Yi\3E>=/):EQ#Y=xj:@gWR,:Cm,6fxEO<uCw-/E?]:jq=N)l#,sKD+N@DUG6GN-b!-y+^ U'hqW92qkUa]T:9Qi 2Rjh^6D\.s4P@1n';Mb}~FtJy+ME,+NQIl'_8Tb?U ce3!5]bj9YgfGKGIJM|ln.'\Y[1^w(anR|QQS)W{B .DQ^JUDzFc<?0+>5<y[mz=bU"=WeMt^76\I"MMbr1S/^yr8Qm?.2uR5xLOD7OG;lOSh
\*#.A.=uWQO#JB':cldQj47:LA]W9[]e9gCotuQJukt?oJ_UwCNY
fdi|_3 6Io2k}>
i!PhSkRBc>_Fqp!l174jgRPR)5EUOdw
bmaJq+i!_Ks<k1:k|kW-7
sKuPD^6keyBgAIgbR'J>,5r9AMl*'l3}'Jsju-7RJ<S~C^;sLn)pYPU>h<|"Wa